# <center> Stacking

In [1]:
%store -r X_train
%store -r X_test

%store -r y_train
%store -r y_test

%store -r X_train_sub
%store -r X_test_sub
%store -r y_train_sub
%store -r y_test_sub

In [2]:
%store -r hp_lgbm
%store -r lgbm_cv
%store -r xgb_model

In [3]:
__da

/Users/maksymsuprunenko/anaconda3/lib/python3.7/site-packages/tqdm/_tqdm.py:604: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Basic Data Analysis tools was loaded


In [20]:
from vecstack import stacking

def create_submission(model, submission_name, X_test, send_submission=False, message="Stacking"):
    result = model.predict(X_test)
    submission = pd.DataFrame({
        "ID": np.arange(result.shape[0]), 
        "item_cnt_month": result
    })
    submission.to_csv(submission_name, index=False)
   
    if send_submission:
        !kaggle competitions submit -c competitive-data-science-predict-future-sales -f {submission_name} -m {message}

In [ ]:
models = [lgbm_cv, xgb_model]
s1_train, s1_test = stacking(models,
                           X_train_sub, y_train_sub, X_test_sub,
                           regression=True,
                           mode='oof_pred_bag',
                           random_state=25,
                           n_folds = 3,
                           verbose=2)

xgb_lev2 = xgb.XGBRegressor(learning_rate=0.04, 
                        n_estimators=5,
                        max_depth=3,
                        n_jobs=-1,
                        random_state=17
                       )

# Fit the 2nd level model on the output of level 1
xgb_lev2.fit(s1_train, y_train_sub)

task:         [regression]
metric:       [mean_absolute_error]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [LGBMRegressor]
    fold  0:  [0.20133565]
    fold  1:  [0.16236561]
    fold  2:  [0.14488234]
    fold  3:  [0.16479533]
    ----
    MEAN:     [0.16834473] + [0.02053797]
    FULL:     [0.16834473]

model  1:     [XGBRegressor]
    fold  0:  [0.18465339]
    fold  1:  [0.16552404]
    fold  2:  [0.14120001]


In [14]:
first_level = pd.DataFrame(xgb_model.predict(X_test), columns=['xgb_model'])
first_level['lgbm_cv'] = lgbm_cv.predict(X_test)
first_level['lgbm_hp'] = hp_lgbm.predict(X_test)

In [19]:
first_level_test = pd.DataFrame(xgb_model.predict(X_test_sub), columns=['xgb_model'])
first_level_test['lgbm_cv'] = lgbm_cv.predict(X_test_sub)
first_level_test['lgbm_hp'] = hp_lgbm.predict(X_test_sub)

In [16]:
import xgboost as xgb

xgb_lev2 = xgb.XGBRegressor(learning_rate=0.04, 
                        n_estimators=2,
                        max_depth=3,
                        n_jobs=-1,
                        random_state=17
                       )

xgb_lev2.fit(first_level, y_test)

[15:35:46] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=2,
             n_jobs=-1, nthread=None, objective='reg:linear', random_state=17,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [21]:
create_submission(xgb_lev2, 'stacking_submission_hp.csv', first_level_test,  send_submission=True)

100%|███████████████████████████████████████| 3.57M/3.57M [00:06<00:00, 575kB/s]
Successfully submitted to Predict Future Sales